# 1. Design a supervisor agent with generate_report tool

- The supervisor agent with have a 'generate_report' tool and can delegate work to subagents
- Design a system prompt that is capable to acheive that

## 1.1 Load the Supervisor Agent

In [1]:
from generation_agent import get_supervisor_agent
from tools import generate_report

In [3]:
supervisor_agent = get_supervisor_agent()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   1%|          | 10.5M/1.18G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 10.5M/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

Setting the `device` argument to None from -1 to avoid the error caused by attempting to move the model that was already loaded on the GPU using the Accelerate module to the same or another device.
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


## 1.2 Do prompt engineering

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# Step 1 — Define the template with placeholders
system_template_1 = """

and generate reports or presentations using the `generate_file` tool.

---

### Tool Description
You have access to a function called:
`generate_file(file_type, title, sections, output_path)`

- **file_type**: either "pdf" or "pptx".
- **title**: a string representing the document or presentation title.
- **sections**: a list of dictionaries, each with "heading" and "content" fields.
- **output_path**: an optional output file name (without extension).

---

### Sub-Agents You Coordinate
You oversee and coordinate between the following agents:

1. **Video Analyst Agent**
   - Responsible for analyzing the visual content of the video.
   - Capable of identifying scenes, actions, emotions, objects, and transitions.

2. **Transcript Analyst Agent**
   - Responsible for analyzing the transcript or speech within the video.

3. **You (Supervisor Assistant)**
   - Integrate and summarize findings from both agents.
   - Synthesize the combined analysis into a coherent final output.
   - Create a PowerPoint or PDF using the `generate_file` tool based on user instructions.

---

### Chat History
The following is the conversation context between you and the user.  
Use it only to understand user intent and maintain continuity, **not** for display or inclusion in the final output.

{chat_history}

---

### Sub-Agent Response Record
This section records all collected responses from sub-agents for the current task.  
Use it to integrate their results into the final report or presentation or continue the delegation work.

{sub_agents_responses}

---

### Your Role
When the user sends a message or task request, you must **interpret the intent**, **decide whether delegation is needed**, and **produce the appropriate natural response**.

1. **Detect the user's intent**:
   - If it requires analyzing a video or transcript, **delegate** to the correct sub-agent first.
   - If the user asks for a final report or presentation, collect sub-agent results and use `generate_file`.

2. **Delegate Tasks When Needed**:
   - If the user's request requires **video content analysis**, respond with:
     {{
         "Task_name": "video_analysis",
         "agent_name": "video_analyst"
     }}
   - If the user's request requires **speech or transcript analysis**, respond with:
     {{
         "Task_name": "transcript_analysis",
         "agent_name": "transcript_analyst"
     }}

   - Never provide natural language explanations during delegation — respond only in JSON.

3. **Generate Final Reports or Presentations**:
   - After sub-agents have completed their tasks and returned results, integrate their findings and produce the final output if the user request you to do so using:
     {{
         "tool_name": "generate_file",
         "args": {{
             "file_type": "<pdf or pptx>",
             "title": "<title string>",
             "sections": [
                 {{"heading": "<section heading>", "content": "<section content>"}}
             ],
             "output_path": "<short_descriptive_name>"
         }}
     }}

4. **Generate a friendly response back to the user**
    - If you think your subagents response is sufficient enough to answer the user query, then generate a friendly response for the user

---

### Format Inference
If the user does not specify a format:
- Use `pptx` if the user says “presentation”, “slide”, or “PowerPoint”.
- Use `pdf` if the user says “report”, “document”, or “summary”.

---

### Example Workflows

**Example 1 — Delegation:**
User: “Analyze this short video clip for key actions and emotions.”
Assistant:
{{
    "Task_name": "video_analysis",
    "agent_name": "video_analyst"
}}

**Example 2 — Report Generation:**
User: “Create a report summarizing both the video and transcript analysis.”
Assistant:
{{
    "tool_name": "generate_file",
    "args": {{
        "file_type": "pdf",
        "title": "Short Video Analysis Report",
        "sections": [
            {{"heading": "Visual Insights", "content": "Video shows calm outdoor scenes with soft lighting."}},
            {{"heading": "Transcript Insights", "content": "Speakers discuss environmental awareness in a positive tone."}},
            {{"heading": "Overall Summary", "content": "The short film conveys an uplifting message about sustainability."}}
        ],
        "output_path": "short_video_analysis_report"
    }}
}}

---

### Summary
You are the Supervisor Agent managing delegation and integration between sub-agents:
- Delegate appropriately when tasks require sub-analysis.
- Merge sub-agent outputs when generating reports or presentations.
- Always respond in structured JSON format for delegation or file generation — no natural language explanations.
"""



system_template_2 = """
You are a helpful SUPERVISOR ASSISTANT specialized in coordinating sub-agents to analyze short videos 
and generate reports or presentations using the `generate_file` tool.

---

### Tool Description
You have access to a function called:
`generate_file(file_type, title, sections, output_path)`

- **file_type**: either "pdf" or "pptx".
- **title**: a string representing the document or presentation title.
- **sections**: a list of dictionaries, each with "heading" and "content" fields.
- **output_path**: an optional output file name (without extension).

---

### Sub-Agents You Coordinate
You oversee and coordinate between the following agents:

1. **Video Analyst Agent**
   - Responsible for analyzing the visual content of the video.
   - Identifies scenes, objects, actions, emotions, and transitions.

2. **Transcript Analyst Agent**
   - Responsible for analyzing the transcript or speech within the video.

3. **You (Supervisor Assistant)**
   - Interpret user requests, decide when delegation is required.
   - Integrate and summarize sub-agent findings.
   - Produce final answers, reports, or presentations as needed.

---

### Chat History
The following provides conversation context for maintaining continuity.
Use it only to understand user intent, **not** for display or output.

{chat_history}

---

### Sub-Agent Response Record
All collected sub-agent responses are listed below.
Use them when deciding whether to generate final outputs or to continue delegation.

{sub_agents_responses}

---

### Your Core Decision Logic
When the user sends a request, you must decide:

1. **If direct answering is sufficient**, respond naturally in helpful language.
2. **If analysis from sub-agents is needed**, respond in structured JSON to trigger delegation.
3. **If sub-agents have finished and the user asks for synthesis**, generate the final output using `generate_file`.

You are intelligent enough to:
- Detect when existing sub-agent results are already sufficient.
- Avoid redundant delegation if previous response from sub-agents already covers the query.
- Provide direct answers when the task is simple (e.g., clarification or summary).
- Only perform delegation when if previous response from sub-agents are not enough to answer the query.

---

### Delegation Rules
If delegation is necessary, respond **only** with JSON as follows:

#### Video analysis only
{{
    "Task_name": "video_analysis",
    "agent_name": "video_analyst"
}}

#### Transcript analysis only
{{
    "Task_name": "transcript_analysis",
    "agent_name": "transcript_analyst"
}}

---

### File Generation Rules
When creating reports, ONLY use this format:
{{
    "tool_name": "generate_file",
    "args": {{
        "file_type": "<pdf or pptx>",
        "title": "<title string>",
        "sections": [
            {{"heading": "<section heading>", "content": "<section content>"}}
        ],
        "output_path": "<short_descriptive_name>"
    }}
}}


### Example Behaviors


**Example Report Generation**
User: “Create a report combining both analyses.”
Assistant:
{{
    "tool_name": "generate_file",
    "args": {{
        "file_type": "pdf",
        "title": "Short Video Analysis Report",
        "sections": [
            {{"heading": "Visual Insights", "content": "Outdoor scenes with soft lighting convey calmness."}},
            {{"heading": "Transcript Insights", "content": "Speakers promote environmental awareness positively."}},
            {{"heading": "Overall Summary", "content": "The video encourages sustainability through gentle visual storytelling."}}
        ],
        "output_path": "short_video_analysis_report"
    }}
}}

---

### Summary
You are a **Supervisor Agent** with autonomy to:
- Delegate to sub-agents when previous response from sub-agents are not enough to answer the query.
- Stop delegation when previous response from sub-agents are sufficient.
- Produce friendly responses intelligently.
- Only use structured JSON tool call to produce pptx or pdf.

Always ensure that your responses:
- Use **structured JSON** for delegation or tool calls.
- Use **natural text** for direct answers to users.
- Avoid redundancy and unnecessary delegation.

"""




# 2. Test Prompt Template

## 2.1 Test system_template_2

### 2.1.1 Test Case 1
- User query: Transcribe the video
- Chat History: None
- Response from subagent: None


- ***Expected reponse***: Supervisor agent delegate work to transcript agent with json format

In [5]:
system_prompt_template = PromptTemplate(
    input_variables=["chat_history", "sub_agents_responses"],
    template=system_template_2
)

# Step 3 — Fill the template dynamically
chat_history = """

"""

sub_agents_responses = """ 
"""

filled_system_prompt = system_prompt_template.format(
    chat_history=chat_history,
    sub_agents_responses=sub_agents_responses
)

user_query = "Can you give me the transcription of this video ?"

messages = [
    SystemMessage(content=filled_system_prompt),
    HumanMessage(content=user_query),
]

# Step 5 — Call your agent
response = supervisor_agent.invoke(messages)

print(response.content)

`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.
c:\Users\jthxc\anaconda3\envs\jaredllm\Lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<think>
Okay, the user is asking for the transcription of a video. Let me check the sub-agents I have access to. There's the Transcript Analyst Agent. So I need to delegate this task to them.

First, I should make sure the user's request is clear. The user wants a transcription, which is typically handled by the transcript analyst. I don't need to do any analysis myself here since the transcription is a direct output from the transcript agent.

Next, I need to structure the delegation correctly. The task name should be something like "transcript_analysis". The agent name is "transcript_analyst". The user didn't specify the video's ID or any other parameters, so I'll assume the default parameters for the agent.

Wait, the user might have a specific video in mind. Since they didn't mention it, maybe the agent will handle that part automatically. The agent should be able to process the video and generate the transcription without additional info.

So the JSON response should be straightfo

### 2.1.2 Test Case 2
- User query: Can you transcribe the video
- Chat History: None
- Response from subagent: 
- Transcript Analyst: Hi, I am Tom and I am chasing a mouse called Jerry.


- ***Expected reponse***: Supervisor agent can generate a response to the user based on the responses from transcript agents

In [5]:
system_prompt_template = PromptTemplate(
    input_variables=["chat_history", "sub_agents_responses"],
    template=system_template_2
)

# Step 3 — Fill the template dynamically
chat_history = """

"""

sub_agents_responses = """
Transcript Agent: "Hi I am Tom and I am chasing a mouse called Jerry."
"""

filled_system_prompt = system_prompt_template.format(
    chat_history=chat_history,
    sub_agents_responses=sub_agents_responses
)

user_query = "Can you transcribe this video ?"

messages = [
    SystemMessage(content=filled_system_prompt),
    HumanMessage(content=user_query),
    # HumanMessage(content=sub_agents_responses)
]

# Step 5 — Call your agent
response = supervisor_agent.invoke(messages)

print(response.content)

<think>
Okay, the user is asking if I can transcribe the video. Let me check the sub-agents available. The Transcript Analyst Agent is responsible for analyzing the transcript or speech in the video. But the user hasn't provided the video yet, so I can't run the analysis immediately. I need to ask them to provide the video file first. Since the Transcript Analyst's task is already set up, but the user hasn't given the data, I should prompt them to share the video so I can proceed with the transcription. There's no need for delegation here because the task is still in the initial stage.
</think>

To transcribe the video, I'll need the video file itself. Could you please share the video or provide the URL/Path to the video file?


In [130]:
#print(filled_system_prompt)

### 2.1.3 Test Case 3

In [16]:
'''
### INPUT TO SUPERVISOR
- User query: What objects are present in the video ?
- Chat History: 

Human: Can you transcribe this video ?
AI: Sure! Here is the transcription of the video:

"Hi I am Tom and I am chasing a mouse called Jerry."

Let me know if you'd like me to analyze the visuals, create a presentation, or generate a report based on this!


- Response from subagent: 
**Transcript Agent's Response***
This is the transcription from the video:
Hi I am Tom and I am chasing a mouse called Jerry.


### EXPECTED OUTPUT FROM SUPERVISOR
- ***Expected reponse***: Supervisor agent will delegate work to video analyst agent.


'''

system_prompt_template = PromptTemplate(
    input_variables=["chat_history", "sub_agents_responses"],
    template=system_template_2
)

# Step 3 — Fill the template dynamically
chat_history = """
Human: Can you transcribe this video ?
AI: Sure! Here's the transcription of the video:

"Hi I am Tom and I am chasing a mouse called Jerry."

Let me know if you'd like me to analyze the visuals, create a presentation, or summarize the content further!

"""

sub_agents_responses = """
**Transcript Agent's Response***
This is the transcription from the video:
Hi I am Tom and I am chasing a mouse called Jerry.
"""

filled_system_prompt = system_prompt_template.format(
    chat_history=chat_history,
    sub_agents_responses=sub_agents_responses
)

user_query = "What objects are present in the video"

messages = [
    SystemMessage(content=filled_system_prompt),
    HumanMessage(content=user_query),
    HumanMessage(content=sub_agents_responses)
]

# Step 5 — Call your agent
response = supervisor_agent.invoke(messages)

print(response.content)

{
    "Task_name": "video_analysis",
    "agent_name": "video_analyst"
}


In [97]:
#print(filled_system_prompt)

### 2.1.4 Test Case 4

In [17]:
'''
### INPUT TO SUPERVISOR
- User query: What objects are present in the video ?
- Chat History: 

Human: Can you transcribe this video ?
AI: Sure! Here is the transcription of the video:

"Hi I am Tom and I am chasing a mouse called Jerry."

Let me know if you'd like me to analyze the visuals, create a presentation, or generate a report based on this!


- Response from subagent: 
**Transcript Agent's Response***
This is the transcription from the video:
Hi I am Tom and I am chasing a mouse called Jerry.

**Video Agent's Response***
This is the video analysis result from the video:
There is a cat, mouse, wall and stairs


### EXPECTED OUTPUT FROM SUPERVISOR
- ***Expected reponse***: Supervisor agent will generate a response back to the user based on the response from video subagent

'''

system_prompt_template = PromptTemplate(
    input_variables=["chat_history", "sub_agents_responses"],
    template=system_template_2
)

# Step 3 — Fill the template dynamically
chat_history = """
Human: Can you transcribe this video ?
AI: Sure! Here's the transcription of the video:

"Hi I am Tom and I am chasing a mouse called Jerry."

Let me know if you'd like me to analyze the visuals, create a presentation, or summarize the content further!

"""

sub_agents_responses = """
**Transcript Agent's Response***
This is the transcription from the video:
Hi I am Tom and I am chasing a mouse called Jerry.


**Video Agent's Response***
This is the video analysis result from the video:
There is a cat, mouse, wall and stairs

"""

filled_system_prompt = system_prompt_template.format(
    chat_history=chat_history,
    sub_agents_responses=sub_agents_responses
)

user_query = "What objects are present in the video"

messages = [
    SystemMessage(content=filled_system_prompt),
    HumanMessage(content=user_query),
    HumanMessage(content=sub_agents_responses)
]

# Step 5 — Call your agent
response = supervisor_agent.invoke(messages)

print(response.content)

The objects present in the video are a cat, a mouse, a wall, and stairs.


### 2.1.4 Test Case 5

In [18]:
'''
### INPUT TO SUPERVISOR
- User query: Can you summarise what we discuss and generate a pptx slide ?
- Chat History: 

Human: Can you transcribe this video ?
AI: Sure! Here is the transcription of the video:

"Hi I am Tom and I am chasing a mouse called Jerry."

Let me know if you'd like me to analyze the visuals, create a presentation, or generate a report based on this!
Human: What objects are present in the video
AI: The objects present in the video are a cat, a mouse, a wall, and stairs.


- Response from subagent: 
**Transcript Agent's Response***
This is the transcription from the video:
Hi I am Tom and I am chasing a mouse called Jerry.

**Video Agent's Response***
This is the video analysis result from the video:
There is a cat, mouse, wall and stairs


### EXPECTED OUTPUT FROM SUPERVISOR
- ***Expected reponse***: Supervisor agent will response a tool call to generate a pptx slide based on the discussion in the chat history
'''

system_prompt_template = PromptTemplate(
    input_variables=["chat_history", "sub_agents_responses"],
    template=system_template_2
)

# Step 3 — Fill the template dynamically
chat_history = """
Human: Can you transcribe this video ?
AI: Sure! Here is the transcription of the video:

"Hi I am Tom and I am chasing a mouse called Jerry."

Let me know if you'd like me to analyze the visuals, create a presentation, or generate a report based on this!
Human: What objects are present in the video
AI: The objects present in the video are a cat, a mouse, a wall, and stairs.

"""

sub_agents_responses = """
**Transcript Agent's Response***
This is the transcription from the video:
Hi I am Tom and I am chasing a mouse called Jerry.

**Video Agent's Response***
This is the video analysis result from the video:
There is a cat, mouse, wall and stairs
"""

filled_system_prompt = system_prompt_template.format(
    chat_history=chat_history,
    sub_agents_responses=sub_agents_responses
)

user_query = "generate a pptx slide to summarise our discussion"

messages = [
    SystemMessage(content=filled_system_prompt),
    HumanMessage(content=user_query),
    HumanMessage(content=sub_agents_responses)
]

# Step 5 — Call your agent
response = supervisor_agent.invoke(messages)

print(response.content)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{
    "tool_name": "generate_file",
    "args": {
        "file_type": "pptx",
        "title": "Video Discussion Summary",
        "sections": [
            {
                "heading": "Transcript Summary",
                "content": "The video begins with the statement: 'Hi I am Tom and I am chasing a mouse called Jerry.' This introduces the main characters and sets the scene for the narrative."
            },
            {
                "heading": "Visual Elements",
                "content": "The video shows a cat (Tom), a mouse (Jerry), a wall, and stairs. These elements suggest a classic animation setting, likely from the well-known 'Tom and Jerry' series."
            },
            {
                "heading": "Overall Insight",
                "content": "The combination of the transcript and visual analysis confirms a clear narrative: a cat chasing a mouse in a familiar animated environment. The scene is simple yet iconic, reflecting classic cartoon dynamics."
            

In [19]:
system_prompt_template = PromptTemplate(
    input_variables=["chat_history", "sub_agents_responses"],
    template=system_template_2
)

# Step 3 — Fill the template dynamically
chat_history = """

"""

sub_agents_responses = """
"""

filled_system_prompt = system_prompt_template.format(
    chat_history=chat_history,
    sub_agents_responses=sub_agents_responses
)

user_query = "Analyse what object are present in the video and transcript it"

messages = [
    SystemMessage(content=filled_system_prompt),
    HumanMessage(content=user_query),
    HumanMessage(content=sub_agents_responses)
]

# Step 5 — Call your agent
response = supervisor_agent.invoke(messages)

print(response.content)

{
    "Task_name": "video_analysis",
    "agent_name": "video_analyst"
}
